In [46]:
pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [51]:
import pandas as pd
import re
import emoji

In [52]:
#read csv
df1 = pd.read_csv('raw_tweets_p1.csv')
#display(df1)
df2 = pd.read_csv('raw_tweets_p2.csv')
#display(df2)
df3 = pd.read_csv('raw_tweets_p3.csv')
df4 = df1.append(df2, ignore_index=True)
df=df4.append(df3, ignore_index=True)
display(df)


,id,time,tweets,emoji
0,1593507164559056897,2022-11-18 07:31:28,@BLACKPINKSTATS5 Love to see it 👏 the numbers ...,👏
1,1593507163971911682,2022-11-18 07:31:28,@PostmanCarson congratulations John 🎊 👏,👏
2,1593507163506462721,2022-11-18 07:31:28,@Tyranjf @RunningWithTum1 @we_are_runners @Ich...,👏
3,1593507132296527873,2022-11-18 07:31:20,A story book scene.. congrats to both of you! 👏,👏
4,1593507130291769345,2022-11-18 07:31:20,@thatpositivepie 👏👏👏 and good look,👏
...,...,...,...,...
300609,1593520889517297668,2022-11-18 08:26:00,@UKDaddyBear1 Needs my tongue 👅,👅
300610,1593520798521778177,2022-11-18 08:25:39,@LoganFoxx69 Wow nice dick 🍆👅,👅
300611,1593520489259204608,2022-11-18 08:24:25,Would you 💋 or 👅?\n\nbokep indo viral mesum li...,👅
300612,1593520348082970624,2022-11-18 08:23:51,@victoriacake3x Yeaaaah👅👅,👅


In [53]:
temp_data = df.groupby("emoji").size().sort_values(ascending = False)
# temp_list=df.index[df['emoji'] == '👅'].tolist()
# print(len(temp_list))
temp_data

emoji
💛        6082
💗        5000
💘        5000
💚        5000
💜        5000
         ... 
💋        5000
💓        5000
💕        5000
👅        4531
emoji       1
Length: 61, dtype: int64

Remove Duplicates

In [54]:
df.drop_duplicates(keep='first', subset="tweets",inplace=True)
df.info()
#df.loc[120].tweets
df=df.reset_index()

#data cleaning process
raw_dataset = df['tweets']
target_emoji = df['emoji']
tweet_id=df['id']
tweet_time=df['time']
#df.drop('index')
#print(raw_dataset)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 283895 entries, 0 to 300517
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      283895 non-null  object
 1   time    283895 non-null  object
 2   tweets  283895 non-null  object
 3   emoji   283895 non-null  object
dtypes: object(4)
memory usage: 10.8+ MB


In [55]:
temp_list=df.index[df['emoji'] == '👅'].tolist()
print(len(temp_list))

4058


In [56]:
display(df)

,index,id,time,tweets,emoji
0,0,1593507164559056897,2022-11-18 07:31:28,@BLACKPINKSTATS5 Love to see it 👏 the numbers ...,👏
1,1,1593507163971911682,2022-11-18 07:31:28,@PostmanCarson congratulations John 🎊 👏,👏
2,2,1593507163506462721,2022-11-18 07:31:28,@Tyranjf @RunningWithTum1 @we_are_runners @Ich...,👏
3,3,1593507132296527873,2022-11-18 07:31:20,A story book scene.. congrats to both of you! 👏,👏
4,4,1593507130291769345,2022-11-18 07:31:20,@thatpositivepie 👏👏👏 and good look,👏
...,...,...,...,...,...
283890,300239,1593564433950834688,2022-11-18 11:19:02,@misssophyxo How sexy the body would lick you💋...,👅
283891,300435,1593541867089567745,2022-11-18 09:49:22,@similolamobee Mama I saw the caption \nI am n...,👅
283892,300440,1593541295082856448,2022-11-18 09:47:05,"A day later, I'm still looking at Mommy's musc...",👅
283893,300467,1593536193651695619,2022-11-18 09:26:49,👅Drop your handles let’s follow you!,👅


Remove hastags, urls, mentions, BOM character, none-English words and extra spaces after cleaning

In [57]:
import nltk 
nltk.download('words')
words = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [58]:
#reference to https://github.com/giuliadnt/emo_tweepy with some changes
def clean_tweets(list_of_tweets):
    """
    :param list_of_tweets: list of unique tweets (no duplicates)
    :return: list of preprocessed tweets
    the function standardizes mentions --> @USER
    and URLs --> @URL
    removes BOM character
    """
    # matches @username
    mentions = re.compile('(@.*?)(?=\s|\:)')#(^|\s)
    # http://stackoverflow.com/questions/6038061/regular-expression-to-find-urls-within-a-string
    links = re.compile('(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?')
    hashtags = re.compile('#([a-zA-Z0-9_]{1,50})')
    #no_mentions = [re.sub(mentions, "", i) for i in list_of_tweets]
    lower_case = [i.lower() for i in list_of_tweets]
    no_mentions = [re.sub(r"@\S+", "", i) for i in lower_case]
    #no_links = [re.sub(links, "", i) for i in no_mentions]
    no_links = [re.sub(r"http\S+", "", i) for i in no_mentions]
    no_links = [re.sub(r"www.\S+", "", i) for i in no_links]
    no_BOM = [re.sub(u"\ufeff", "", i) for i in no_links]
    #no_hash = [re.sub(hashtags,"",i)for i in no_BOM]
    no_hash = [re.sub(r"#\S+","",i)for i in no_BOM]
    no_newlinechar = [re.sub('\n', "", i) for i in no_hash]
    #no_nonEnglish = [" ".join(w for w in nltk.wordpunct_tokenize(sent) if w.lower() in words or not w.isalpha()) for sent in no_newlinechar]
    # for sent in no_newlinechar:
    #   for word in sent:
    #     word = " ".join(w for w in nltk.wordpunct_tokenize(word) if w.lower() in words or not w.isalpha())
    no_extraspace = [" ".join(i.split()) for i in no_newlinechar]
    return no_extraspace

In [59]:
temp=["I need you girl 넌 아름다워I need you girl 너무 차가워I need you girl I need you girlI need you girl I need you girl - I NEED U -BTS Favorite K-Pop Artist","Yeeeeeeeeeeees.Happy Birthday @Tilly"]
clean_tweets(temp)
# sent = " ".join(w for w in nltk.wordpunct_tokenize(temp) if w.lower() in words or not w.isalpha())
# print(sent)

['i need you girl 넌 아름다워i need you girl 너무 차가워i need you girl i need you girli need you girl i need you girl - i need u -bts favorite k-pop artist',
 'yeeeeeeeeeeees.happy birthday']

In [60]:
#remove hashtags,urls,@,extraspances..
cleaned_dataset=clean_tweets(raw_dataset)
#display(cleaned_dataset)

In [61]:
#remove emoji from tweets
cleaned_tweets=[emoji.replace_emoji(i,replace="") for i in cleaned_dataset]

In [31]:
#display(cleaned_tweets)

In [62]:
#organize post and emoji to a dataframe
data2 = {'id':tweet_id,
         'time':tweet_time,
         'tweets':cleaned_tweets,
        'emoji':target_emoji}

Emojimotion_df = pd.DataFrame(data2)
display(Emojimotion_df)



,id,time,tweets,emoji
0,1593507164559056897,2022-11-18 07:31:28,love to see it the numbers they both pull is ...,👏
1,1593507163971911682,2022-11-18 07:31:28,congratulations john,👏
2,1593507163506462721,2022-11-18 07:31:28,well done champ,👏
3,1593507132296527873,2022-11-18 07:31:20,a story book scene.. congrats to both of you!,👏
4,1593507130291769345,2022-11-18 07:31:20,and good look,👏
...,...,...,...,...
283890,1593564433950834688,2022-11-18 11:19:02,how sexy the body would lick you,👅
283891,1593541867089567745,2022-11-18 09:49:22,mama i saw the caption i am not willing to glo...,👅
283892,1593541295082856448,2022-11-18 09:47:05,"a day later, i'm still looking at mommy's muscles",👅
283893,1593536193651695619,2022-11-18 09:26:49,drop your handles let’s follow you!,👅


Drop the sentences that are less than 4 words

In [63]:
ind_list=[]
ind=0
temp=cleaned_tweets
for tw in temp:
  tw_word_list = re. findall(r"[\w'\"]+|[,.!?] ", tw)
  #print(len(tw_word))
  if len(tw_word_list)<4:
    ind_list.append(ind)
  ind=ind+1
Emojimotion_df.drop(ind_list,axis=0,inplace=True)

In [64]:
Emojimotion_df=Emojimotion_df.reset_index()
display(Emojimotion_df)

,index,id,time,tweets,emoji
0,0,1593507164559056897,2022-11-18 07:31:28,love to see it the numbers they both pull is ...,👏
1,3,1593507132296527873,2022-11-18 07:31:20,a story book scene.. congrats to both of you!,👏
2,5,1593507124596084736,2022-11-18 07:31:19,"four arms more like... ""for us""",👏
3,7,1593507111023034369,2022-11-18 07:31:15,here here. an amazing team of reps delivering ...,👏
4,8,1593507062893641728,2022-11-18 07:31:04,they should do that to all cex.,👏
...,...,...,...,...,...
222858,283890,1593564433950834688,2022-11-18 11:19:02,how sexy the body would lick you,👅
222859,283891,1593541867089567745,2022-11-18 09:49:22,mama i saw the caption i am not willing to glo...,👅
222860,283892,1593541295082856448,2022-11-18 09:47:05,"a day later, i'm still looking at mommy's muscles",👅
222861,283893,1593536193651695619,2022-11-18 09:26:49,drop your handles let’s follow you!,👅


In [65]:
#write the finnal dataframe to a csv file
Emojimotion_df.to_csv('/content/cleaned_tweets.csv',index=False)

In [66]:
#some dataset testing 
import seaborn as sns
import matplotlib.pyplot as plt

In [67]:
data = Emojimotion_df.groupby("emoji").size().sort_values(ascending = False)
data

emoji
➖     4636
💓     4517
😕     4386
💩     4381
👇     4331
😬     4293
▶️    4291
🎧     4284
🔴     4261
😠     4258
😡     4242
😓     4235
💀     4233
💰     4184
😞     4177
📍     4172
😔     4171
😣     4165
😫     4154
☁️    4118
😝     4112
😮     4052
😄     4051
😳     3977
❗️    3898
💛     3876
⚡️    3852
😹     3849
😌     3792
😃     3756
💃     3731
💖     3718
✊     3692
💙     3691
🌷     3583
✅     3573
✔️    3488
😋     3458
🙏     3449
😚     3432
✨     3363
🎁     3318
💗     3317
👑     3308
💕     3306
🌈     3298
💚     3281
💜     3266
💘     3257
😘     3248
💋     3207
🍃     3136
👏     3113
😻     3039
🌹     3007
🌺     2953
👅     2907
👌     2808
⬅️    2624
☕️    2588
dtype: int64

In [ ]:
# sns.set(style="whitegrid", color_codes=True)
# data = Emojimotion_df.groupby("emoji").size().sort_values(ascending = False)
# pal = sns.color_palette("Blues_d", len(data))
# sns.barplot(x=data.index, y=data, palette=pal)

# plt.show()

# data